<a href="https://colab.research.google.com/github/TiWeu/Traffic-Sign-Detection/blob/main/yolov3_darknet_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the darknet
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 19.21 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.
/content/darknet


In [2]:
# change makefile to have GPU and OPENCV enable
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make
!./darknet

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

In [3]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74
!ls

--2023-12-10 21:13:43--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  37.6MB/s    in 4.1s    

2023-12-10 21:13:48 (38.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]

3rdparty	darknet			docker-compose.yml     LICENSE	      results
backup		darknet53.conv.74	Dockerfile.cpu	       Makefile       scripts
build		DarknetConfig.cmake.in	Dockerfile.gpu	       net_cam_v3.sh  src
build.ps1	darknet_images.py	image_yolov3.sh        net_cam_v4.sh  vcpkg.json
cfg		darknet.py		image_yolov4.sh        obj	      vcpkg.json.opencv23
cmake		darknet_video.py	include		       package.xml    video_yolov3.sh
CMakeLists.txt	data			json_mjpeg_streams.sh  README.md      video_yolov4.sh

In [4]:
!./darknet detect cfg/yolov3.cfg weights/yolov3.weights data/test-image.jpg

 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  

In [5]:
# Install and unzip dataset
import zipfile, urllib.request, shutil
url = "https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip"
file_name = 'FullIJCNN2013.zip'

with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(file_name) as zf:
        zf.extractall()

In [6]:
import os
import pandas as pd
import cv2

In [7]:
# Defining lists for categories according to the classes ID's
# Prohibitory category:
# circular Traffic Signs with white background and red border line
p = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]

# Danger category:
# triangular Traffic Signs with white background and red border line
d = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

# Mandatory category:
# circular Traffic Signs with blue background
m = [33, 34, 35, 36, 37, 38, 39, 40]

# Other category:
o = [6, 12, 13, 14, 17, 32, 41, 42]

In [8]:
ann = pd.read_csv("FullIJCNN2013" + '/' + 'gt.txt',
                  names=['ImageID', 'XMin', 'YMin', 'XMax', 'YMax', 'ClassID'],
                  sep=';')
ann.head()

,ImageID,XMin,YMin,XMax,YMax,ClassID
0,00000.ppm,774,411,815,446,11
1,00001.ppm,983,388,1024,432,40
2,00001.ppm,386,494,442,552,38
3,00001.ppm,973,335,1031,390,13
4,00002.ppm,892,476,1006,592,39


In [9]:
# Adding new empty columns to dataFrame to save numbers for YOLO format
ann['CategoryID'] = ''
ann['center x'] = ''
ann['center y'] = ''
ann['width'] = ''
ann['height'] = ''

# Getting category's ID according to the class's ID
ann.loc[ann['ClassID'].isin(p), 'CategoryID'] = 0
ann.loc[ann['ClassID'].isin(d), 'CategoryID'] = 1
ann.loc[ann['ClassID'].isin(m), 'CategoryID'] = 2
ann.loc[ann['ClassID'].isin(o), 'CategoryID'] = 3

# Calculating bounding box's center in x and y for all rows
ann['center x'] = (ann['XMax'] + ann['XMin']) / 2
ann['center y'] = (ann['YMax'] + ann['YMin']) / 2

# Calculating bounding box's width and height for all rows
ann['width'] = ann['XMax'] - ann['XMin']
ann['height'] = ann['YMax'] - ann['YMin']

In [10]:
# By using copy() we create separate dataFrame and initial dataFrame will not be changed.
r = ann.loc[:, ['ImageID',
                'CategoryID',
                'center x',
                'center y',
                'width',
                'height']].copy()
r.head()

,ImageID,CategoryID,center x,center y,width,height
0,00000.ppm,1,794.5,428.5,41,35
1,00001.ppm,2,1003.5,410.0,41,44
2,00001.ppm,2,414.0,523.0,56,58
3,00001.ppm,3,1002.0,362.5,58,55
4,00002.ppm,2,949.0,534.0,114,116


In [11]:
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00


In [12]:
%pdb on

Automatic pdb calling has been turned ON


In [13]:
path_dir = "/content/darknet/FullIJCNN2013/"
# path = os.chdir(path_dir)
path = os.listdir(path_dir)

for file in path:

    # Checking if filename ends with '.ppm'
    if file.endswith('.ppm'):
        file_path = '/content/darknet/FullIJCNN2013/' + file

        # Reading image and getting its real width and height
        image_ppm = cv2.imread(file_path)

        # Slicing from tuple only first two elements
        h, w = image_ppm.shape[:2]

        # Slicing only name of the file without extension
        image_name = file[:-4]

        sub_r = r.loc[r['ImageID'] == file].copy()

        # Normalizing calculated bounding boxes' coordinates according to the real image width and height
        sub_r['center x'] = sub_r['center x'] / w
        sub_r['center y'] = sub_r['center y'] / h
        sub_r['width'] = sub_r['width'] / w
        sub_r['height'] = sub_r['height'] / h

        resulted_frame = sub_r.loc[:, ['CategoryID',
                                           'center x',
                                           'center y',
                                           'width',
                                           'height']].copy()

        # Checking if there is no any annotations for current image
        if resulted_frame.isnull().values.all():
            # Skipping this image
            continue

        # Saving resulted Pandas dataFrame into txt file
        path_to_save = path_dir + image_name + '.txt'
        if not os.path.exists(path_to_save):
          resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')

        # Saving image in jpg format by OpenCV function
        path_to_save = path_dir + image_name + '.jpg'
        if not os.path.exists(path_to_save):
          cv2.imwrite(path_to_save, image_ppm)

In [14]:
for file in path:
    # Checking if filename ends with '.ppm'
    if file.endswith('.ppm'):
      file_path = path_dir + file
      os.remove(file_path)

In [38]:
path = os.listdir(path_dir)

# Defining list to write paths in
p = []

for file in path:
    # Checking if filename ends with '.jpg'
    if file.endswith('.jpg'):
        # Preparing path to save into train.txt file
        path_to_save_into_txt_files = path_dir + file
        p.append(path_to_save_into_txt_files + '\n')


# Slicing first 15% of elements from the list to write into the test.txt file
p_test = p[:int(len(p) * 0.15)]

# Write into the train.txt file rest of the elements
p_train = p[int(len(p) * 0.15):]


# Creating file train.txt and writing 85% of lines in it
with open('train.txt', 'w') as train_txt:
    for e in p_train:
        # Writing current path at the end of the file
        train_txt.write(e)
    print("TRAIN DATA CREATED!")
    print(p_train)

# Creating file test.txt and writing 15% of lines in it
with open('test.txt', 'w') as test_txt:
    for e in p_test:
        # Writing current path at the end of the file
        test_txt.write(e)
    print("TEST DATA CREATED!")

/content/darknet/FullIJCNN2013/00516.jpg
/content/darknet/FullIJCNN2013/00199.jpg
/content/darknet/FullIJCNN2013/00091.jpg
/content/darknet/FullIJCNN2013/00291.jpg
/content/darknet/FullIJCNN2013/00284.jpg
/content/darknet/FullIJCNN2013/00860.jpg
/content/darknet/FullIJCNN2013/00346.jpg
/content/darknet/FullIJCNN2013/00212.jpg
/content/darknet/FullIJCNN2013/00654.jpg
/content/darknet/FullIJCNN2013/00283.jpg
/content/darknet/FullIJCNN2013/00290.jpg
/content/darknet/FullIJCNN2013/00822.jpg
/content/darknet/FullIJCNN2013/00195.jpg
/content/darknet/FullIJCNN2013/00457.jpg
/content/darknet/FullIJCNN2013/00180.jpg
/content/darknet/FullIJCNN2013/00039.jpg
/content/darknet/FullIJCNN2013/00846.jpg
/content/darknet/FullIJCNN2013/00486.jpg
/content/darknet/FullIJCNN2013/00886.jpg
/content/darknet/FullIJCNN2013/00358.jpg
/content/darknet/FullIJCNN2013/00466.jpg
/content/darknet/FullIJCNN2013/00047.jpg
/content/darknet/FullIJCNN2013/00287.jpg
/content/darknet/FullIJCNN2013/00041.jpg
/content/darknet

In [39]:
NAME_COUNTER = 0
categories = ["prohibitory", "danger", "mandatory", "other"]
with open(path_dir + 'classes.names', 'w') as names:
    for cat in categories:
        names.write(cat + '\n')  # Copying all info from file txt to names
NUM_CATEGORY = len(categories)
NUM_CATEGORY

4

In [40]:
data_path = "/content/darknet/cfg/"
with open(data_path + 'ts_data.data', 'w') as data:
    # Writing needed 5 lines
    # Number of classes
    # By using '\n' we move to the next line
    data.write('classes = ' + str(NUM_CATEGORY) + '\n')

    # Location of the train.txt file
    data.write('train = ' + path_dir + 'train.txt' + '\n')

    # Location of the test.txt file
    data.write('valid = ' + path_dir + 'test.txt' + '\n')

    # Location of the classes.names file
    data.write('names = ' + path_dir + 'classes.names' + '\n')

    # Location where to save weights
    data.write('backup = backup')

In [41]:
%cd darknet
!ls

[Errno 20] Not a directory: 'darknet'
/content/darknet
3rdparty	darknet53.conv.74	Dockerfile.gpu	       Makefile       src
backup		DarknetConfig.cmake.in	FullIJCNN2013	       net_cam_v3.sh  test.txt
build		darknet_images.py	FullIJCNN2013.zip      net_cam_v4.sh  train.txt
build.ps1	darknet.py		image_yolov3.sh        obj	      vcpkg.json
cfg		darknet_video.py	image_yolov4.sh        package.xml    vcpkg.json.opencv23
cmake		data			include		       README.md      video_yolov3.sh
CMakeLists.txt	docker-compose.yml	json_mjpeg_streams.sh  results	      video_yolov4.sh
darknet		Dockerfile.cpu		LICENSE		       scripts


# Zu ändern

/darknet/cfg/yolov3.cfg:

- batch=64
- subdivision=64

In [ ]:
# Start the training
!./darknet detector train cfg/ts_data.data cfg/yolov3.cfg darknet53.conv.74 -dont_show

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.231367), count: 1, class_loss = 2197.210449, iou_loss = 2.924316, total_loss = 2200.134766 
 total_bbox = 2125, rewritten_bbox = 0.047059 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 159.825638, iou_loss = 0.000000, total_loss = 159.825638 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.358328), count: 1, class_loss = 524.133179, iou_loss = 1.538269, total_loss = 525.671448 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.352945), count: 2, class_loss = 2218.915039, iou_loss = 4.655029, total_loss = 2223.570068 
 total_bbox = 2128, rewritten_bbox = 0.046992 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 161.475327, iou_loss = 0.00

In [25]:
!./darknet detector demo cfg/ts_data.data cfg/yolov3-voc.cfg weights/yolov3_train_1000.weights data/traffic-sign-to-test.mp4 -out_filename traffic-sign-to-test.avi -dont_show

 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
Demo
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: